In [1]:
import math
import numpy as np
#golden search
def bracket(f,x1,h):
    c = 1.618033989 
    f1 = f(x1)
    x2 = x1 + h; f2 = f(x2)
  # Determine downhill direction and change sign of h if needed
    if f2 > f1:
        h = -h
        x2 = x1 + h; f2 = f(x2)
      # Check if minimum between x1 - h and x1 + h
        if f2 > f1: return x2,x1 - h 
  # Search loop
    for i in range (100):    
        h = c*h
        x3 = x2 + h; f3 = f(x3)
        if f3 > f2: return x1,x3
        x1 = x2; x2 = x3
        f1 = f2; f2 = f3
    print("Bracket did not find a mimimum")

def search(f,a,b,tol=1.0e-9):
    nIter = int(math.ceil(-2.078087*math.log(tol/abs(b-a))))
    R = 0.618033989
    C = 1.0 - R
  # First telescoping
    x1 = R*a + C*b; x2 = C*a + R*b
    f1 = f(x1); f2 = f(x2)
  # Main loop
    for i in range(nIter):
        if f1 > f2:
            a = x1
            x1 = x2; f1 = f2
            x2 = C*a + R*b; f2 = f(x2)
        else:
            b = x2
            x2 = x1; f2 = f1
            x1 = R*a + C*b; f1 = f(x1)
    if f1 < f2: return x1,f1
    else: return x2,f2
#powell
def powell(F,x,h=0.1,tol=1.0e-6):
    
    def f(s): return F(x + s*v)    # F in direction of v

    n = len(x)                     # Number of design variables
    df = np.zeros(n)               # Decreases of F stored here
    u = np.identity(n)             # Vectors v stored here by rows
    for j in range(30):            # Allow for 30 cycles:
        xOld = x.copy()            # Save starting point
        fOld = F(xOld)
      # First n line searches record decreases of F
        for i in range(n):
            v = u[i]
            a,b = bracket(f,0.0,h)
            s,fMin = search(f,a,b)
            df[i] = fOld - fMin
            fOld = fMin
            x = x + s*v
      # Last line search in the cycle    
        v = x - xOld
        a,b = bracket(f,0.0,h)
        s,fLast = search(f,a,b)
        x = x + s*v
      # Check for convergence
        if math.sqrt(np.dot(x-xOld,x-xOld)/n) < tol: return x,j+1
      # Identify biggest decrease & update search directions
        iMax = np.argmax(df)
        for i in range(iMax,n-1):
            u[i] = u[i+1]
        u[n-1] = v
    print("Powell did not converge")
    

In [11]:
#高斯消去法(之前作業的)
def Gauss_elimination(a,b):
    A=a.tolist() 
    B=b.tolist()
    n = len(A)
    B_Orgin=[0] * n
    
    for i in range(1,n):
        for j in range(i,n): 
            G = A[j][i-1]/A[i-1][i-1]
            
            for k in range(i-1,n): 
                A[j][k] = A[j][k] - A[i-1][k]* G
                
            B[j] = B[j]-B[i-1]*G

    for q in range(0,n):
        B_Orgin[q]=B[q]
        
    B[n-1] = B[n-1]/A[n-1][n-1]
    
    
    for i in range(n-2,-1,-1):
        for j in range(n-1,i,-1):
            B[i] = B[i]- A[i][j]*B[j]
        B[i] = B[i]/A[i][i]                 
     
    B_array=np.zeros([len(B_Orgin),1])
    for t in range(0,len(B_Orgin)):
        B_array[t][0]=B_Orgin[t]
    B_List=B_array.tolist()
    
    UT_array=augMatrix(A,B_List) 
    X = B
    #回傳B
    return np.array(X)

    
    



 
    #增廣矩陣
def augMatrix(x, y):
    return [AA + bb for AA, bb in zip(x,y)]

In [14]:
def f(x):
    global v, weight, penalty
    #設所有參數為一
    P = 1
    E = 1
    L = 1
     #δ
    Delta = 1
    
    lamda = 100
    #2*2^1/2
    c = 2*math.sqrt(2)
    #x[0] = A1, x[1] = A2, x[2] = A3
    A_a = np.array([[c*x[1]+x[2], -1*x[2], x[2]],\
                [-1*x[2], x[2], -1*x[2]],\
                [x[2], -1*x[2], c*x[0]+x[2]]])
    #𝐸 /2*2^1/2*L
    A = A_a*E/(c*L)
    b = np.array([0, -P, 0])
    #call 高斯消去法求解
    v = Gauss_elimination(A,b)
    #F(x) 
    weight = P*L**2/(E*Delta)*(x[0] + x[1] + math.sqrt(2)*x[2])
    #懲罰函數P(x)
    penalty = (max(0, abs(v[1])-1))**2 + (max(0, -x[0]))**2 + \
    (max(0, -x[1]))**2 + (max(0, -x[2]))**2
    #merit function:F(x) + lamda*P(x)
    return weight + lamda*penalty

    

In [15]:

xStart = np.array([1.0, 1.0, 1.0])
x,numIter = powell(f, xStart)

print("x = ", x)
print("v = ", v)
print("Relative weight F = ", weight)
print("Number of cycles = ", numIter)


x =  [3.73870376 3.73870366 5.28732564]
v =  [-0.26747239 -1.06988953 -0.26747238]
Relative weight F =  14.95481504706307
Number of cycles =  10
